# Introduction to Feature Engineering

[Resource](https://www.learndatasci.com/tutorials/intro-feature-engineering-machine-learning-python/)

During the creation of my first linear regression models, I noticed a serious knowledge gap in the form of feature engineering. After some preparation, a new learning path has been formed, and I will take the time to do a much needed deep dive into feature engineering. This is an intermediary step in the creation of my first legitimate model. Not the most fun topic, but like all others, it's an essential one.

As I always say, let's begin.

# Introduction

Let's put a face to the name: **Feature engineering is the process of transforming data to increase the predictive performance of machine learning models**. it's both useful and necessary for the following reasons:
1. Often better predictive accuracy: Leads to better weighting of variables. It can even lead to faster convergence.
1. Better interpretability of relationships in the data: When we engineer new features and understand how they relate with our outcome of interest, that opens up our understanding of the data. If we skip feature engineering and use complex models (that to a large degree automate feature engineering), we may still achieve a high evaluation score, at the cost of better understanding out data and its relationship with the target variable.

Every data science pipeline begins with **Exploratory Data Analysis (EDA)**. EDA is a crucial pre-cursor step as we get a better sense of what features we need to create/modify. The next step is usually data cleaning/standardization depending on how unstructured or messy the data is.

Feature engineering follows next and we begin that process by evaluating the baseline performance of the data at hand. We then iteratively construct features and continuously evaluate model performance (and compare it with the baseline performance) through a process called feature selection, until we are satisfied with the results.

## What this article does and doesn't cover

Feature engineering is a vast field as there are many domain-specific tangents. This article covers some of the popular techniques employed in handling tabular datasets. This will not cover feature engineering for natural language processing, image classification, time-series data, etc.

(Unfortunate. I don't. care for any of these except time-series data.)

# The two approaches to feature engineering

1. **The checklist approach:** using tried and tested methods to construct features.
1. **The domain-based approach:** incorporating domain knowledge of the dataset's subject matter into constructing new features.

Now we're gonna take a look at these approaches using some actual data. Let'do some importing.

In [74]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set_palette(sns.color_palette(['#851836', '#edbd17']))
sns.set_style("darkgrid")

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

We will now demonstrate the checklist approach using a dataset on supermarket sales. Note that the dataset has been slightly modified for the tutorial.

In [75]:
df = pd.read_csv('data/supermarket_sales.csv')
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,1:08:00 PM,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29:00 AM,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,1:23:00 PM,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,8:33:00 PM,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37:00 AM,Ewallet,604.17,4.761905,30.2085,5.3


# The Checklist Approach

## Numeric Aggregations

Numeric aggregation is a common feature engineering approach for longitudinal or panel data (data where subjects are repeated). In our dataset, we have categorical variables with repeated observations.

**Numeric aggregation involves three parameters:**
1. Categorical column
1. Numeric column(s) to be aggregated
1. Aggregation type: Mean, median, mode, standard deviation, variance, count, etc

We're gonna cover three examples of numeric aggregations based on mean, standard deviation, and count respectively. Our three parameters are:
1. Branch - categorical column, which we're grouping by
1. Tax 5%, Unit Price, Product line, and Gender - numeric columns to be aggregated
1. Mean, standard deviation, and count - aggregations to be used on the numeric columns

Let's start by grouping by Branch and perform the three statistical aggregations by transforming the numeric columns of interest.

In [76]:
grouped_df = df.groupby('Branch')

df[['tax_branch_mean', 'unit_price_mean']] = grouped_df[['Tax 5%', 'Unit price']].transform('mean')
df[['tax_branch_std', 'unit_price_std']] = grouped_df[['Tax 5%', 'Unit price']].transform('std')
df[['product_count', 'gender_count']] = grouped_df[['Product line', 'Gender']].transform('count')

And we see the features we've just created below.

(I added `Invoice ID` for more clarity.)

In [77]:
df[['Invoice ID', 'Branch', 'tax_branch_mean', 'unit_price_mean', 'tax_branch_std',
    'unit_price_std', 'product_count', 'gender_count']].head(10)



,Invoice ID,Branch,tax_branch_mean,unit_price_mean,tax_branch_std,unit_price_std,product_count,gender_count
0,750-67-8428,Alex,14.874001,54.780853,11.030477,26.132127,340,340
1,226-31-3081,Giza,16.052367,56.609024,12.531470,27.179799,328,328
2,631-41-3108,Alex,14.874001,54.780853,11.030477,26.132127,340,340
3,123-19-1176,Alex,14.874001,54.780853,11.030477,26.132127,340,340
4,373-73-7910,Alex,14.874001,54.780853,11.030477,26.132127,340,340
5,699-14-3026,Giza,16.052367,56.609024,12.531470,27.179799,328,328
6,355-53-5943,Alex,14.874001,54.780853,11.030477,26.132127,340,340
7,315-22-5665,Giza,16.052367,56.609024,12.531470,27.179799,328,328
8,665-32-9167,Alex,14.874001,54.780853,11.030477,26.132127,340,340
9,692-92-5582,Cairo,15.232024,55.659277,11.545122,26.224559,332,332


**Note:** The reason you're seeing repeating Branches after the grouping is because we used `.transform()`, not `.aggregate()`. This way we have one row per transaction with all its features, rather than losing the majority of the data.

## Choosing numeric aggregation parameters

How do we pick which three parameters to use? Well, that will depend on your domain knowledge and your understanding of the dataset. For example, in this dataset, if you feel like the variation in the **average** (aggregation type) **Rating** (numeric variable) based on the **Branch** (categorical column) is important in predicting **gross income** (target variable), create the feature! If you feel like the count of the products in the Product Line, by branch, is important in informing gross income, encode that as a feature!

Now if you can test as many combinations of the three parameters - go ahead - as long as you are meticulous at selecting only those features that have enough predictive power i,e, be sure to have a rigorous feature selection process.

Below we can see a couple of the columns we created (`tax_branch_mean` and `unit_price_mean`). They are aggregations based on the `Branch` variable.

In [78]:
df[['Tax 5%', 'Unit price', 'Branch', 'tax_branch_mean', 'unit_price_mean']]

,Tax 5%,Unit price,Branch,tax_branch_mean,unit_price_mean
0,26.1415,74.69,Alex,14.874001,54.780853
1,3.8200,15.28,Giza,16.052367,56.609024
2,16.2155,46.33,Alex,14.874001,54.780853
3,23.2880,58.22,Alex,14.874001,54.780853
4,30.2085,86.31,Alex,14.874001,54.780853
...,...,...,...,...,...
995,2.0175,40.35,Giza,16.052367,56.609024
996,48.6900,97.38,Cairo,15.232024,55.659277
997,1.5920,31.84,Alex,14.874001,54.780853
998,3.2910,65.82,Alex,14.874001,54.780853


## But why is all of this necessary?

Aren't good models designed to take aggregation into account? To an extent, yes, but not always. It depends a lot on the size and dimensionality of your dataset. The larger the dataset, the more features (by several orders of magnitude) you can create. When there are too many features, the model has too many competing signals to predict the target variable.

Feature engineering tries to explicitly focus the model's attention on certain features. To summarize, feature engineering is not about creating "new" information, but rather directing and/or focusing the model's attention on certain information, that you as the data scientist judge to be important.

# Indicator Variables and Interaction Terms

Following the same patten of thinking as numeric aggregations, we can construct indicator variables and interaction terms.

**Indicator variables** only take on the value 0 or 1 to indicate the absence or presence of some information.

For example, we're about to define an indicator variable `unit_price_50` to indicate if the product has unit price greater than 50. To put it into perspective, think of an e-commerce store having free shipping on all orders above $50; this may be useful information in predicting customer behavior and worth an explicit definition for the model.

**Interaction terms** are created base on the presence of interaction effects between two or more variables. This is largely driven by domain expertise, although there are statistical tests to help determine them (beyond the scope of this article. Definitely investigate this in-depth later). For example, while free shipping may affect customer rating, free shipping combined with quantity may have a different effect on customer rating, which would be useful to encode (assuming customer rating is the target variable in this case). Below we define the variable `unit_price_50 * qty` to be exactly that.

We use `np.where()` to create an indicator variable `unit_price_50` that encodes 1 when unit price is above 50 and 0 otherwise.

In [79]:
df['unit_price_50'] = np.where(df['Unit price'] > 50, 1, 0)
df['unit_price_50 * qty'] = df['unit_price_50'] * df['Quantity']

df[['Invoice ID', 'unit_price_50', 'unit_price_50 * qty']].head()

,Invoice ID,unit_price_50,unit_price_50 * qty
0,750-67-8428,1,7
1,226-31-3081,0,0
2,631-41-3108,0,0
3,123-19-1176,1,8
4,373-73-7910,1,7
